# PRC LFP lithium-ion battery LCI data and LCIA script

#### Tara Greig's thesis LCA model
________________________________________________________________________________________________________________




In [ ]:
# PACKAGE IMPORT #

import pandas as pd
import openpyxl
import xlrd
from pycel import ExcelCompiler
import olca_ipc as ipc
import olca_schema as o
from datetime import datetime

import uuid
import math
from typing import Callable

from uuid import uuid4
import numpy as np
import json

params = {'mathtext.default': 'regular' }

client = ipc.Client(8080)
client

### LCI Data
Please see the excel files for LCI process data. Citations are included indicating sources of LCI and or data. The thesis indicates alterations, exchanges, and assumptions made in the LCIs.

In [ ]:
### DATA IMPORT ###

df_lioh = pd.read_excel("Data/BattProcess_PRC.xlsx", sheet_name='lioh')
df_graphite = pd.read_excel("Data/BattProcess_PRC.xlsx", sheet_name='graphite')
df_lfp = pd.read_excel("Data/BattProcess_PRC.xlsx", sheet_name='lfp')
df_cathode = pd.read_excel("Data/BattProcess_PRC.xlsx", sheet_name='cathode')
df_anode = pd.read_excel("Data/BattProcess_PRC.xlsx", sheet_name='anode')
df_electrolyte = pd.read_excel("Data/BattProcess_PRC.xlsx", sheet_name='electrolyte')
df_separator = pd.read_excel("Data/BattProcess_PRC.xlsx", sheet_name='separator')
df_cell = pd.read_excel("Data/BattProcess_PRC.xlsx", sheet_name='cell')
df_module = pd.read_excel("Data/BattProcess_PRC.xlsx", sheet_name='module')
df_batt_parts = pd.read_excel("Data/BattProcess_PRC.xlsx", sheet_name='batt_parts')
df_bss = pd.read_excel("Data/BattProcess_PRC.xlsx", sheet_name='bss')


In [ ]:
# LIST, VARIABLE, DICTIONARY CREATION #

processes_names = [

"Concentrated spodumene",
"Lithium hydroxide",

"Graphite ore",
"Graphite concentrate",
"Spherical graphite",
"Spherical purified graphite",
"Coated spherical graphite",

"Lithium iron phosphate powder",

"Cathode current collector",
"Cathode paste",
"Cathode",

"Anode current collector",
"Anode paste",
"Anode",

"Ethylene carbonate",
"Lithium hexafluorophosphate",
"Lithium fluoride",
"Phosphorous pentachloride",
"Electrolyte",

"Separator",

"LFP cell",
"Cell container",
"Tab aluminium",
"Tab copper",
"Multilayer pouch",

"LFP module",
"BMS",

"Inverter",
"Charger",
"System controller",
"BMS 200",

"Housing",
"BSS LFP"
]



len(processes_names)


### Major processing steps

Create processes in OpenLCA software

- Lithium carbonate processing
- Lithium hydroxide processing
- LFP powder synthesis, hydrothermal
- Cathode manufacture
- Anode manufacture
- Electrolyte manufacture
- Separator manufacture
- Cell assembly
- Module assembly
- BSS parts
- Battery Storage System assembly


In [ ]:
### LIOH ###

def lioh_lci(scenario_num):
    spod_lioh_steps = ["Concentrated spodumene","Lithium hydroxide"]
    
    for x in spod_lioh_steps:
        process = o.Process() #creates an empty process
        process.name = x + '_' + str(scenario_num)
        process.process_type = o.ProcessType.UNIT_PROCESS #LCI_RESULT for system processes
        df = df_lioh.copy()
        df = df[df['step'] == x]

        if spod_lioh_steps.index(x) > 0:
            #as this process is interconnected with the previous one we created above, we search the one we just created within Ecoinvent in OpenLCA and take its UUID
            predecessor_product = spod_lioh_steps[spod_lioh_steps.index(x)-1] + '_' + str(scenario_num)

            process_descriptor = client.get(o.Process, name = predecessor_product)
            ids = process_descriptor.id

            df.loc[df['flow name'] == spod_lioh_steps[spod_lioh_steps.index(x)-1], 'provider UUID'] = ids


        #assigns the creation date/time to the description part of the process
        dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))
        process.description = 'Added as new process from the olca-ipc python API on %s. For lithium hydroxide from spodumene' % (dt_object)
        
        exchange_list = []
        for index, row in df.iterrows():
            
            flow = client.get(o.Flow, str(row['UUID dataset'])) #defines the flows according to the UUIDs we sent in LCI
            
            exchange = o.Exchange()
            
            if str(row["in/out"]) == "in": #checks&identified whether flow is input or output
                exchange.is_input = True
            else:
                exchange.is_input = False
                
            exchange.flow =  flow
            
            for unit_group in client.get_all(o.UnitGroup): #downloads all units from ecoinvent and checks&matches our unit from our LCI
                for unit in unit_group.units:
                    if unit.name == row["unit"]:
                        exchange.unit = unit
                for flow_property in client.get_all(o.FlowProperty):
                    if row["flow property"]== flow_property.name:
                        exchange.flow_property = flow_property

            exchange.amount = row["amount"] #defines the amount according to the LCI
            
            if row["is reference"] == 1: #checks the inventory and defines the reference flow. reference flow would be your main output (e.g. Laterite ore from mining)
                exchange.is_quantitative_reference = True
            else:
                exchange.is_quantitative_reference = False
                
            if str(row["provider UUID"]) != "":  #checks provider UUIDs. if it is empty in the inventory (if it is a elementary flow), it leaves it empty, otherwise it fills it out accordingly
                exchange.default_provider = client.get(o.Process ,str(row["provider UUID"]))
            
            if str(row["tech"]) == "avoided": #checks whether flow is avoided product or not.
                exchange.is_avoided_product = True
            else:
                exchange.is_avoided_product = False
                
            exchange_list.append(exchange)
        process.exchanges=exchange_list
        client.put(process)


In [ ]:
### GRAPHITE ###

graphite_lci_steps = [
    "Graphite ore",
    "Graphite concentrate",
    "Spherical graphite",
    "Spherical purified graphite",
    "Coated spherical graphite"
]

def graphite_lci(scenario_num):
    # make LICO3 and predecessors
    for x in graphite_lci_steps:
        process = o.Process() #creates an empty process
        process.name = x + '_' + scenario_num
        process.process_type = o.ProcessType.UNIT_PROCESS #LCI_RESULT for system processes
        df = df_graphite.copy()
        df = df[df['step'] == x]

        #df_readout = df_readout.fillna("")

        if graphite_lci_steps.index(x) > 0:
            #as this process is interconnected with the previous one we created above, we search the one we just created within Ecoinvent in OpenLCA and take its UUID
            predecessor_product = graphite_lci_steps[graphite_lci_steps.index(x)-1] + '_' + scenario_num

            process_descriptor = client.get(o.Process, name = predecessor_product)
            ids = process_descriptor.id

            df.loc[df['flow name'] == graphite_lci_steps[graphite_lci_steps.index(x)-1], 'provider UUID'] = ids
        
        #assigns the creation date/time to the description part of the process
        dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))
        process.description = 'Added as new process from the olca-ipc python API on %s. Graphite processes' % (dt_object)
        exchange_list = []
        for index, row in df.iterrows():
            
            flow = client.get(o.Flow, str(row['UUID dataset'])) #defines the flows according to the UUIDs we sent in LCI
            
            exchange = o.Exchange()
            
            if str(row["in/out"]) == "in": #checks&identified whether flow is input or output
                exchange.is_input = True
            else:
                exchange.is_input = False
                
            exchange.flow =  flow
            
            for unit_group in client.get_all(o.UnitGroup): #downloads all units from ecoinvent and checks&matches our unit from our LCI
                for unit in unit_group.units:
                    if unit.name == row["unit"]:
                        exchange.unit = unit
                for flow_property in client.get_all(o.FlowProperty):
                    if row["flow property"]== flow_property.name:
                        exchange.flow_property = flow_property

            exchange.amount = row["amount"] #defines the amount according to the LCI
            
            if row["is reference"] == 1: #checks the inventory and defines the reference flow. reference flow would be your main output (e.g. Laterite ore from mining)
                exchange.is_quantitative_reference = True
            else:
                exchange.is_quantitative_reference = False
                
            if str(row["provider UUID"]) != "":  #checks provider UUIDs. if it is empty in the inventory (if it is a elementary flow), it leaves it empty, otherwise it fills it out accordingly
                exchange.default_provider = client.get(o.Process ,str(row["provider UUID"]))
            
            if str(row["tech"]) == "avoided": #checks whether flow is avoided product or not.
                exchange.is_avoided_product = True
            else:
                exchange.is_avoided_product = False
                
            exchange_list.append(exchange)
        process.exchanges=exchange_list
        client.put(process)

In [ ]:
### LFP POWDER ###

def lfp_lci(scenario_num):
    name = "Lithium iron phosphate powder"
    process = o.Process() #creates an empty process
    process.name = name + '_' + scenario_num
    process.process_type = o.ProcessType.UNIT_PROCESS #LCI_RESULT for system processes
    df = df_lfp.copy()
    df = df[df['step'] == name]
    #df_readout = df_readout.fillna("")
    process_descriptor = client.get(o.Process, name = "Lithium hydroxide_" + scenario_num)
    ids = process_descriptor.id
    df.loc[df['flow name'] == 'Lithium hydroxide',"provider UUID"] = ids


    #assigns the creation date/time to the description part of the process
    dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))
    process.description = 'Added as new process from the olca-ipc python API on %s. LFP powder from brines or spodumene.' % (dt_object)

    exchange_list = []
    for index, row in df.iterrows():
        
        flow = client.get(o.Flow, str(row['UUID dataset'])) #defines the flows according to the UUIDs we sent in LCI
        
        exchange = o.Exchange()
        
        if str(row["in/out"]) == "in": #checks&identified whether flow is input or output
            exchange.is_input = True
        else:
            exchange.is_input = False
            
        exchange.flow =  flow
        
        for unit_group in client.get_all(o.UnitGroup): #downloads all units from ecoinvent and checks&matches our unit from our LCI
            for unit in unit_group.units:
                if unit.name == row["unit"]:
                    exchange.unit = unit
            for flow_property in client.get_all(o.FlowProperty):
                if row["flow property"]== flow_property.name:
                    exchange.flow_property = flow_property

        exchange.amount = row["amount"] #defines the amount according to the LCI
        
        if row["is reference"] == 1: #checks the inventory and defines the reference flow. reference flow would be your main output (e.g. Laterite ore from mining)
            exchange.is_quantitative_reference = True
        else:
            exchange.is_quantitative_reference = False
            
        if str(row["provider UUID"]) != "":  #checks provider UUIDs. if it is empty in the inventory (if it is a elementary flow), it leaves it empty, otherwise it fills it out accordingly
            exchange.default_provider = client.get(o.Process ,str(row["provider UUID"]))
        
        if str(row["tech"]) == "avoided": #checks whether flow is avoided product or not.
            exchange.is_avoided_product = True
        else:
            exchange.is_avoided_product = False
            
        exchange_list.append(exchange)
    process.exchanges=exchange_list
    client.put(process)

In [ ]:
### CATHODE ###

cathode_lci_steps = [
    "Cathode current collector",
    "Cathode paste",
    "Cathode"
]

def cathode_lci(scenario_num):
    # make LICO3 and predecessors
    for x in cathode_lci_steps:
        process = o.Process() #creates an empty process
        process.name = x + '_' + scenario_num
        process.process_type = o.ProcessType.UNIT_PROCESS #LCI_RESULT for system processes
        df = df_cathode.copy()
        df = df[df['step'] == x]
        #df_readout = df_readout.fillna("")

        if x == "Cathode paste":
            process_descriptor = client.get(o.Process, name = "Lithium iron phosphate powder_" + scenario_num)
            ids = process_descriptor.id
            df.loc[df['flow name'] == 'Lithium iron phosphate powder',"provider UUID"] = ids

        elif x == 'Cathode':
            #as this process is interconnected with the previous one we created above, we search the one we just created within Ecoinvent in OpenLCA and take its UUID
            process_descriptor1 = client.get(o.Process, name = 'Cathode paste_' + scenario_num)
            process_descriptor2 = client.get(o.Process, name = 'Cathode current collector_' + scenario_num)
            ids1 = process_descriptor1.id
            ids2 = process_descriptor2.id

            df.loc[df['flow name'] == 'Cathode paste', 'provider UUID'] = ids1
            df.loc[df['flow name'] == 'Cathode current collector', 'provider UUID'] = ids2
        
        #assigns the creation date/time to the description part of the process
        dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))
        process.description = 'Added as new process from the olca-ipc python API on %s. Cathode processes.' % (dt_object)
        
        exchange_list = []
        for index, row in df.iterrows():
            
            flow = client.get(o.Flow, str(row['UUID dataset'])) #defines the flows according to the UUIDs we sent in LCI
            
            exchange = o.Exchange()
            
            if str(row["in/out"]) == "in": #checks&identified whether flow is input or output
                exchange.is_input = True
            else:
                exchange.is_input = False
                
            exchange.flow =  flow
            
            for unit_group in client.get_all(o.UnitGroup): #downloads all units from ecoinvent and checks&matches our unit from our LCI
                for unit in unit_group.units:
                    if unit.name == row["unit"]:
                        exchange.unit = unit
                for flow_property in client.get_all(o.FlowProperty):
                    if row["flow property"]== flow_property.name:
                        exchange.flow_property = flow_property

            exchange.amount = row["amount"] #defines the amount according to the LCI
            
            if row["is reference"] == 1: #checks the inventory and defines the reference flow. reference flow would be your main output (e.g. Laterite ore from mining)
                exchange.is_quantitative_reference = True
            else:
                exchange.is_quantitative_reference = False
                
            if str(row["provider UUID"]) != "":  #checks provider UUIDs. if it is empty in the inventory (if it is a elementary flow), it leaves it empty, otherwise it fills it out accordingly
                exchange.default_provider = client.get(o.Process ,str(row["provider UUID"]))
            
            if str(row["tech"]) == "avoided": #checks whether flow is avoided product or not.
                exchange.is_avoided_product = True
            else:
                exchange.is_avoided_product = False
                
            exchange_list.append(exchange)
        process.exchanges=exchange_list
        client.put(process)

In [ ]:
### ANODE ###

anode_lci_steps = [
    "Anode current collector",
    "Anode paste",
    "Anode"
]

def anode_lci(scenario_num):
    # make LICO3 and predecessors
    for x in anode_lci_steps:
        process = o.Process() #creates an empty process
        process.name = x + '_' + scenario_num
        process.process_type = o.ProcessType.UNIT_PROCESS #LCI_RESULT for system processes
        df = df_anode.copy()
        df = df[df['step'] == x]
        #df_readout = df_readout.fillna("")

        if x == "Anode paste":
            process_descriptor = client.get(o.Process, name = "Coated spherical graphite_" + scenario_num)
            ids = process_descriptor.id
            df.loc[df['flow name'] == 'Coated spherical graphite',"provider UUID"] = ids

        elif x == 'Anode':
            #as this process is interconnected with the previous one we created above, we search the one we just created within Ecoinvent in OpenLCA and take its UUID
            process_descriptor1 = client.get(o.Process, name = 'Anode paste_' + scenario_num)
            process_descriptor2 = client.get(o.Process, name = 'Anode current collector_' + scenario_num)
            ids1 = process_descriptor1.id
            ids2 = process_descriptor2.id

            df.loc[df['flow name'] == 'Anode paste', 'provider UUID'] = ids1
            df.loc[df['flow name'] == 'Anode current collector', 'provider UUID'] = ids2

        #assigns the creation date/time to the description part of the process
        dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))
        process.description = 'Added as new process from the olca-ipc python API on %s. Anode processes.' % (dt_object)
        
        exchange_list = []
        for index, row in df.iterrows():
            
            flow = client.get(o.Flow, str(row['UUID dataset'])) #defines the flows according to the UUIDs we sent in LCI
            
            exchange = o.Exchange()
            
            if str(row["in/out"]) == "in": #checks&identified whether flow is input or output
                exchange.is_input = True
            else:
                exchange.is_input = False
                
            exchange.flow =  flow
            
            for unit_group in client.get_all(o.UnitGroup): #downloads all units from ecoinvent and checks&matches our unit from our LCI
                for unit in unit_group.units:
                    if unit.name == row["unit"]:
                        exchange.unit = unit
                for flow_property in client.get_all(o.FlowProperty):
                    if row["flow property"]== flow_property.name:
                        exchange.flow_property = flow_property

            exchange.amount = row["amount"] #defines the amount according to the LCI
            
            if row["is reference"] == 1: #checks the inventory and defines the reference flow. reference flow would be your main output (e.g. Laterite ore from mining)
                exchange.is_quantitative_reference = True
            else:
                exchange.is_quantitative_reference = False
                
            if str(row["provider UUID"]) != "":  #checks provider UUIDs. if it is empty in the inventory (if it is a elementary flow), it leaves it empty, otherwise it fills it out accordingly
                exchange.default_provider = client.get(o.Process ,str(row["provider UUID"]))
            
            if str(row["tech"]) == "avoided": #checks whether flow is avoided product or not.
                exchange.is_avoided_product = True
            else:
                exchange.is_avoided_product = False
                
            exchange_list.append(exchange)
        process.exchanges=exchange_list
        client.put(process)

In [ ]:
### ELECTROLYTE ###

electrolyte_lci_steps = [
    "Ethylene carbonate",
    "Lithium fluoride",
    "Phosphorous pentachloride",
    "Lithium hexafluorophosphate",
    "Electrolyte"
]

def electrolyte_lci(scenario_num):
    # make LICO3 and predecessors
    for x in electrolyte_lci_steps:
        process = o.Process() #creates an empty process
        process.name = x + '_' + scenario_num
        process.process_type = o.ProcessType.UNIT_PROCESS #LCI_RESULT for system processes
        df = df_electrolyte.copy()
        df = df[df['step'] == x]
        #df_readout = df_readout.fillna("")
        
        if x == "Lithium hexafluorophosphate":
            process_descriptor1 = client.get(o.Process, name = 'Lithium fluoride_' + scenario_num)
            process_descriptor2 = client.get(o.Process, name = 'Phosphorous pentachloride_' + scenario_num)

            ids1 = process_descriptor1.id
            ids2 = process_descriptor2.id

            df.loc[df['flow name'] == 'Lithium fluoride', 'provider UUID'] = ids1
            df.loc[df['flow name'] == 'Phosphorous pentachloride', 'provider UUID'] = ids2
        
        elif x == 'Electrolyte':
            process_descriptor1 = client.get(o.Process, name = 'Lithium hexafluorophosphate_' + scenario_num)
            process_descriptor2 = client.get(o.Process, name = 'Ethylene carbonate_' + scenario_num)

            ids1 = process_descriptor1.id
            ids2 = process_descriptor2.id

            df.loc[df['flow name'] == 'Lithium hexafluorophosphate', 'provider UUID'] = ids1
            df.loc[df['flow name'] == 'Ethylene carbonate', 'provider UUID'] = ids2

        #assigns the creation date/time to the description part of the process
        dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))
        process.description = 'Added as new process from the olca-ipc python API on %s. Electrolyte processes.' % (dt_object)
        
        exchange_list = []
        for index, row in df.iterrows():
            
            flow = client.get(o.Flow, str(row['UUID dataset'])) #defines the flows according to the UUIDs we sent in LCI
            
            exchange = o.Exchange()
            
            if str(row["in/out"]) == "in": #checks&identified whether flow is input or output
                exchange.is_input = True
            else:
                exchange.is_input = False
                
            exchange.flow =  flow
            
            for unit_group in client.get_all(o.UnitGroup): #downloads all units from ecoinvent and checks&matches our unit from our LCI
                for unit in unit_group.units:
                    if unit.name == row["unit"]:
                        exchange.unit = unit
                for flow_property in client.get_all(o.FlowProperty):
                    if row["flow property"]== flow_property.name:
                        exchange.flow_property = flow_property

            exchange.amount = row["amount"] #defines the amount according to the LCI
            
            if row["is reference"] == 1: #checks the inventory and defines the reference flow. reference flow would be your main output (e.g. Laterite ore from mining)
                exchange.is_quantitative_reference = True
            else:
                exchange.is_quantitative_reference = False
                
            if str(row["provider UUID"]) != "":  #checks provider UUIDs. if it is empty in the inventory (if it is a elementary flow), it leaves it empty, otherwise it fills it out accordingly
                exchange.default_provider = client.get(o.Process ,str(row["provider UUID"]))
            
            if str(row["tech"]) == "avoided": #checks whether flow is avoided product or not.
                exchange.is_avoided_product = True
            else:
                exchange.is_avoided_product = False
                
            exchange_list.append(exchange)
        process.exchanges=exchange_list
        client.put(process)

In [ ]:
### SEPARATOR ###

def separator_lci(scenario_num):
    name = "Separator"
    process = o.Process() #creates an empty process
    process.name = name + '_' + scenario_num
    process.process_type = o.ProcessType.UNIT_PROCESS #LCI_RESULT for system processes
    df = df_separator.copy()
    df = df[df['step'] == name]
    #df_readout = df_readout.fillna("")

    #assigns the creation date/time to the description part of the process
    dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))
    process.description = 'Added as new process from the olca-ipc python API on %s. Separator processes.' % (dt_object)

    exchange_list = []
    for index, row in df.iterrows():
        
        flow = client.get(o.Flow, str(row['UUID dataset'])) #defines the flows according to the UUIDs we sent in LCI
        
        exchange = o.Exchange()
        
        if str(row["in/out"]) == "in": #checks&identified whether flow is input or output
            exchange.is_input = True
        else:
            exchange.is_input = False
            
        exchange.flow =  flow
        
        for unit_group in client.get_all(o.UnitGroup): #downloads all units from ecoinvent and checks&matches our unit from our LCI
            for unit in unit_group.units:
                if unit.name == row["unit"]:
                    exchange.unit = unit
            for flow_property in client.get_all(o.FlowProperty):
                if row["flow property"]== flow_property.name:
                    exchange.flow_property = flow_property

        exchange.amount = row["amount"] #defines the amount according to the LCI
        
        if row["is reference"] == 1: #checks the inventory and defines the reference flow. reference flow would be your main output (e.g. Laterite ore from mining)
            exchange.is_quantitative_reference = True
        else:
            exchange.is_quantitative_reference = False
            
        if str(row["provider UUID"]) != "":  #checks provider UUIDs. if it is empty in the inventory (if it is a elementary flow), it leaves it empty, otherwise it fills it out accordingly
            exchange.default_provider = client.get(o.Process ,str(row["provider UUID"]))
        
        if str(row["tech"]) == "avoided": #checks whether flow is avoided product or not.
            exchange.is_avoided_product = True
        else:
            exchange.is_avoided_product = False
            
        exchange_list.append(exchange)
    process.exchanges=exchange_list
    client.put(process)

In [ ]:
### CELL ###


cell_lci_steps = [
    "Multilayer pouch",
    "Tab aluminium",
    "Tab copper",
    "Cell container",
    "LFP cell"
]

def cell_lci(scenario_num):
    # make LICO3 and predecessors
    for x in cell_lci_steps:
        process = o.Process() #creates an empty process
        process.name = x + '_' + scenario_num
        process.process_type = o.ProcessType.UNIT_PROCESS #LCI_RESULT for system processes
        df = df_cell.copy()
        df = df[df['step'] == x]
        #df_readout = df_readout.fillna("")

        if x == "Cell container":
            process_descriptor1 = client.get(o.Process, name = "Tab aluminium_" + scenario_num)
            process_descriptor2 = client.get(o.Process, name = "Tab copper_" + scenario_num)
            process_descriptor3 = client.get(o.Process, name = "Multilayer pouch_" + scenario_num)


            ids1 = process_descriptor1.id
            ids2 = process_descriptor2.id
            ids3 = process_descriptor3.id


            df.loc[df['flow name'] == 'Tab aluminium',"provider UUID"] = ids1
            df.loc[df['flow name'] == 'Tab copper',"provider UUID"] = ids2
            df.loc[df['flow name'] == 'Multilayer pouch',"provider UUID"] = ids3

        elif x == "LFP cell":
            process_descriptor1 = client.get(o.Process, name = "Cathode_" + scenario_num)
            process_descriptor2 = client.get(o.Process, name = "Anode_" + scenario_num)
            process_descriptor3 = client.get(o.Process, name = "Separator_" + scenario_num)
            process_descriptor4 = client.get(o.Process, name = "Electrolyte_" + scenario_num)
            process_descriptor5 = client.get(o.Process, name = "Cell container_" + scenario_num)


            ids1 = process_descriptor1.id
            ids2 = process_descriptor2.id
            ids3 = process_descriptor3.id
            ids4 = process_descriptor4.id
            ids5 = process_descriptor5.id

            df.loc[df['flow name'] == 'Cathode',"provider UUID"] = ids1
            df.loc[df['flow name'] == 'Anode',"provider UUID"] = ids2
            df.loc[df['flow name'] == 'Separator',"provider UUID"] = ids3
            df.loc[df['flow name'] == 'Electrolyte',"provider UUID"] = ids4
            df.loc[df['flow name'] == 'Cell container',"provider UUID"] = ids5
        
        #assigns the creation date/time to the description part of the process
        dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))
        process.description = 'Added as new process from the olca-ipc python API on %s. LFP Cell, pouch.' % (dt_object)
        
        exchange_list = []
        for index, row in df.iterrows():
            
            flow = client.get(o.Flow, str(row['UUID dataset'])) #defines the flows according to the UUIDs we sent in LCI
            
            exchange = o.Exchange()
            
            if str(row["in/out"]) == "in": #checks&identified whether flow is input or output
                exchange.is_input = True
            else:
                exchange.is_input = False
                
            exchange.flow =  flow
            
            for unit_group in client.get_all(o.UnitGroup): #downloads all units from ecoinvent and checks&matches our unit from our LCI
                for unit in unit_group.units:
                    if unit.name == row["unit"]:
                        exchange.unit = unit
                for flow_property in client.get_all(o.FlowProperty):
                    if row["flow property"]== flow_property.name:
                        exchange.flow_property = flow_property

            exchange.amount = row["amount"] #defines the amount according to the LCI
            
            if row["is reference"] == 1: #checks the inventory and defines the reference flow. reference flow would be your main output (e.g. Laterite ore from mining)
                exchange.is_quantitative_reference = True
            else:
                exchange.is_quantitative_reference = False
                
            if str(row["provider UUID"]) != "":  #checks provider UUIDs. if it is empty in the inventory (if it is a elementary flow), it leaves it empty, otherwise it fills it out accordingly
                exchange.default_provider = client.get(o.Process ,str(row["provider UUID"]))
            
            if str(row["tech"]) == "avoided": #checks whether flow is avoided product or not.
                exchange.is_avoided_product = True
            else:
                exchange.is_avoided_product = False
                
            exchange_list.append(exchange)
        process.exchanges=exchange_list
        client.put(process)

In [ ]:
### MODULE ###


module_lci_steps = [
    "BMS",
    "LFP module"
]

def module_lci(scenario_num):
    # make LICO3 and predecessors
    for x in module_lci_steps:
        process = o.Process() #creates an empty process
        process.name = x + '_' + scenario_num
        process.process_type = o.ProcessType.UNIT_PROCESS #LCI_RESULT for system processes
        df = df_module.copy()
        df = df[df['step'] == x]
        #df_readout = df_readout.fillna("")

        if x == 'LFP module':
            process_descriptor1 = client.get(o.Process, name = "LFP cell_" + scenario_num)
            process_descriptor2 = client.get(o.Process, name = "BMS_" + scenario_num)

            ids1 = process_descriptor1.id
            ids2 = process_descriptor2.id

            df.loc[df['flow name'] == 'LFP cell',"provider UUID"] = ids1
            df.loc[df['flow name'] == 'BMS',"provider UUID"] = ids2
    

        #assigns the creation date/time to the description part of the process
        dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))
        process.description = 'Added as new process from the olca-ipc python API on %s. LFP module, pouch.' % (dt_object)
        
        exchange_list = []
        for index, row in df.iterrows():
            
            flow = client.get(o.Flow, str(row['UUID dataset'])) #defines the flows according to the UUIDs we sent in LCI
            
            exchange = o.Exchange()
            
            if str(row["in/out"]) == "in": #checks&identified whether flow is input or output
                exchange.is_input = True
            else:
                exchange.is_input = False
                
            exchange.flow =  flow
            
            for unit_group in client.get_all(o.UnitGroup): #downloads all units from ecoinvent and checks&matches our unit from our LCI
                for unit in unit_group.units:
                    if unit.name == row["unit"]:
                        exchange.unit = unit
                for flow_property in client.get_all(o.FlowProperty):
                    if row["flow property"]== flow_property.name:
                        exchange.flow_property = flow_property

            exchange.amount = row["amount"] #defines the amount according to the LCI
            
            if row["is reference"] == 1: #checks the inventory and defines the reference flow. reference flow would be your main output (e.g. Laterite ore from mining)
                exchange.is_quantitative_reference = True
            else:
                exchange.is_quantitative_reference = False
                
            if str(row["provider UUID"]) != "":  #checks provider UUIDs. if it is empty in the inventory (if it is a elementary flow), it leaves it empty, otherwise it fills it out accordingly
                exchange.default_provider = client.get(o.Process ,str(row["provider UUID"]))
            
            if str(row["tech"]) == "avoided": #checks whether flow is avoided product or not.
                exchange.is_avoided_product = True
            else:
                exchange.is_avoided_product = False
                
            exchange_list.append(exchange)
        process.exchanges=exchange_list
        client.put(process)

In [ ]:
### BATTERY PARTS ###


batt_parts_lci_steps = [
    "Inverter",
    "Charger",
    "BMS 200",
    "System controller",
]

def batt_parts_lci(scenario_num):
    # make LICO3 and predecessors
    for x in batt_parts_lci_steps:
        process = o.Process() #creates an empty process
        process.name = x + '_' + scenario_num
        process.process_type = o.ProcessType.UNIT_PROCESS #LCI_RESULT for system processes
        df = df_batt_parts.copy()
        df = df[df['step'] == x]
        #df_readout = df_readout.fillna("")

        # if batt_parts_lci_steps.index(x) > 0:
        #     #as this process is interconnected with the previous one we created above, we search the one we just created within Ecoinvent in OpenLCA and take its UUID
        #     predecessor_product = batt_parts_lci_steps[batt_parts_lci_steps.index(x)-1] + '_' + str(scenario_num)

        if x == "System controller":
            process_descriptor = client.get(o.Process, name = 'BMS 200_' + scenario_num)

            ids = process_descriptor.id

            df.loc[df['flow name'] == 'BMS 200',"provider UUID"] = ids

        #     process_descriptor = client.get(o.Process, name = predecessor_product)
        #     ids = process_descriptor.id

        #     df.loc[df['flow name'] == predecessor_product, 'provider UUID'] = ids
        
        #assigns the creation date/time to the description part of the process
        dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))
        process.description = 'Added as new process from the olca-ipc python API on %s. Module parts.' % (dt_object)
        
        exchange_list = []
        for index, row in df.iterrows():
            
            flow = client.get(o.Flow, str(row['UUID dataset'])) #defines the flows according to the UUIDs we sent in LCI
            
            exchange = o.Exchange()
            
            if str(row["in/out"]) == "in": #checks&identified whether flow is input or output
                exchange.is_input = True
            else:
                exchange.is_input = False
                
            exchange.flow =  flow
            
            for unit_group in client.get_all(o.UnitGroup): #downloads all units from ecoinvent and checks&matches our unit from our LCI
                for unit in unit_group.units:
                    if unit.name == row["unit"]:
                        exchange.unit = unit
                for flow_property in client.get_all(o.FlowProperty):
                    if row["flow property"]== flow_property.name:
                        exchange.flow_property = flow_property

            exchange.amount = row["amount"] #defines the amount according to the LCI
            
            if row["is reference"] == 1: #checks the inventory and defines the reference flow. reference flow would be your main output (e.g. Laterite ore from mining)
                exchange.is_quantitative_reference = True
            else:
                exchange.is_quantitative_reference = False
                
            if str(row["provider UUID"]) != "":  #checks provider UUIDs. if it is empty in the inventory (if it is a elementary flow), it leaves it empty, otherwise it fills it out accordingly
                exchange.default_provider = client.get(o.Process ,str(row["provider UUID"]))
            
            if str(row["tech"]) == "avoided": #checks whether flow is avoided product or not.
                exchange.is_avoided_product = True
            else:
                exchange.is_avoided_product = False
                
            exchange_list.append(exchange)
        process.exchanges=exchange_list
        client.put(process)


In [ ]:
### BATTERY STORAGE SYSTEM ###


bss_lci_steps = [
    "Housing",
    "BSS LFP"
]

def bss_lci(scenario_num):
    # make LICO3 and predecessors
    for x in bss_lci_steps:
        process = o.Process() #creates an empty process
        process.name = x  + '_' + scenario_num
        process.process_type = o.ProcessType.UNIT_PROCESS #LCI_RESULT for system processes
        df = df_bss.copy()
        df = df[df['step'] == x]

        if x == "BSS LFP":
            process_descriptor1 = client.get(o.Process, name = "LFP module_" + scenario_num)
            process_descriptor2 = client.get(o.Process, name = "System controller_" + scenario_num)
            process_descriptor3 = client.get(o.Process, name = "Charger_" + scenario_num)
            process_descriptor4 = client.get(o.Process, name = "Inverter_" + scenario_num)
            process_descriptor5 = client.get(o.Process, name = 'Housing_' + scenario_num)

            ids1 = process_descriptor1.id
            ids2 = process_descriptor2.id
            ids3 = process_descriptor3.id
            ids4 = process_descriptor4.id
            ids5 = process_descriptor5.id

            df.loc[df['flow name'] == 'LFP module',"provider UUID"] = ids1
            df.loc[df['flow name'] == 'System controller',"provider UUID"] = ids2
            df.loc[df['flow name'] == 'Charger',"provider UUID"] = ids3
            df.loc[df['flow name'] == 'Inverter',"provider UUID"] = ids4
            df.loc[df['flow name'] == 'Housing', 'provider UUID'] = ids5

            print('BSS LFP_' + scenario_num)  
                  
        #assigns the creation date/time to the description part of the process
        dt_object = datetime.fromtimestamp(datetime.timestamp(datetime.now()))
        process.description = 'Added as new process from the olca-ipc python API on %s. LFP BSS processes.' % (dt_object)
        
        exchange_list = []
        for index, row in df.iterrows():
            
            flow = client.get(o.Flow, str(row['UUID dataset'])) #defines the flows according to the UUIDs we sent in LCI
            
            exchange = o.Exchange()
            
            if str(row["in/out"]) == "in": #checks&identified whether flow is input or output
                exchange.is_input = True
            else:
                exchange.is_input = False
                
            exchange.flow =  flow
            
            for unit_group in client.get_all(o.UnitGroup): #downloads all units from ecoinvent and checks&matches our unit from our LCI
                for unit in unit_group.units:
                    if unit.name == row["unit"]:
                        exchange.unit = unit
                for flow_property in client.get_all(o.FlowProperty):
                    if row["flow property"]== flow_property.name:
                        exchange.flow_property = flow_property

            exchange.amount = row["amount"] #defines the amount according to the LCI
            
            if row["is reference"] == 1: #checks the inventory and defines the reference flow. reference flow would be your main output (e.g. Laterite ore from mining)
                exchange.is_quantitative_reference = True
            else:
                exchange.is_quantitative_reference = False
                
            if str(row["provider UUID"]) != "":  #checks provider UUIDs. if it is empty in the inventory (if it is a elementary flow), it leaves it empty, otherwise it fills it out accordingly
                exchange.default_provider = client.get(o.Process ,str(row["provider UUID"]))
            
            if str(row["tech"]) == "avoided": #checks whether flow is avoided product or not.
                exchange.is_avoided_product = True
            else:
                exchange.is_avoided_product = False
                
            exchange_list.append(exchange)
        process.exchanges=exchange_list
        client.put(process)


### LCIA for different scenarios

See excel sheet for the different scenarios (lithium sourcing, graphite sourcing, and battery manufacturing locations)

Create product systems in OpenLCA software

In [ ]:
x = 'CN'

lioh_lci(scenario_num=x)
graphite_lci(scenario_num=x)
lfp_lci(scenario_num=x)
cathode_lci(scenario_num=x)
anode_lci(scenario_num=x)
electrolyte_lci(scenario_num=x)
separator_lci(scenario_num=x)
cell_lci(scenario_num=x)
module_lci(scenario_num=x)
batt_parts_lci(scenario_num=x)
bss_lci(scenario_num=x)


process_ref = client.get(o.Process, name = "BSS LFP_CN")
config = o.LinkingConfig(prefer_unit_processes=False, provider_linking = o.ProviderLinking.PREFER_DEFAULTS) # system process, for unit process set prefer_unit_processes to True
product_system = client.create_product_system(process_ref, config) #this creates the product system



    # impact_method = client.get(o.ImpactMethod,'2e5cd15d-d539-3141-a950-56d75df9d579') #recipe midpoint 2016
    
    # setup = o.CalculationSetup(target=product_system, impact_method = impact_method, amount = 1) #amount is in kg    
    # result = client.calculate(setup)
    # result.wait_until_ready()
  
    # impact_name = []
    # impact_value = []
    # impact_unit = []
    # process_results = []

    
    # for impact in result.get_total_impacts():
    #     # impact_string = '%s: %s %s\n' % (impact.impact_category.name, impact.amount, impact.impact_category.ref_unit)
    #     # print(impact_string)

    #     impacts = "%s" %(impact.impact_category.name)
    #     impact_name.append(impacts)
    #     value = "%s" %(impact.amount)
    #     impact_value.append(value)
    #     unit = "%s" %(impact.impact_category.ref_unit)
    #     impact_unit.append(unit)

    # output = pd.DataFrame(list(zip(impact_name, impact_value, impact_unit,process_results))) 
    # output.to_excel("Data/Results/scenario_"+str(i)+'_impacts.xlsx')


    # impact_direct = []

    # for impact in result.get_direct_impacts_of():
    #     impact_direct.append(impact)

    # output = pd.DataFrame(list(zip(impact_name, impact_value, impact_unit,process_results))) 
    # output.to_excel("Data/Results/scenario_"+str(i)+'_process_impacts.xlsx')

    








In [ ]:
# code to delete processes, use prior to every new run to delete old processes in OpenLCA

# for j in processes_names:
#     process_name = j + "_CN"
#     process = client.get(o.Process, name = process_name)
#     client.delete(process)